In [24]:
import pandas as pd
from transformers import AutoConfig, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split

In [ ]:
model_path = ""

In [14]:
config = AutoConfig.from_pretrained()
model = AutoModelForSeq2SeqLM.from_config(config)

TypeError: AutoConfig.from_pretrained() missing 1 required positional argument: 'pretrained_model_name_or_path'

In [21]:
test_df = pd.read_csv("./data/datasets/test.csv")
test_df

,artist,song,tag,content,tag_2
0,5 Seconds of Summer,No Shame,[Verse 1: Luke],"Angel, with the gun in your hand\r\nPointin' m...",[Verse]
1,5 Seconds of Summer,No Shame,"[Pre-Chorus: Luke, Luke & Ashton]",Go on and light me like a cigarette\r\nEven if...,[Pre-Chorus]
2,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin'\r\nN...,[Chorus]
3,5 Seconds of Summer,No Shame,[Verse 2: Luke],Lay me in the palm of your hand\r\nI'll give y...,[Verse]
4,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin'\r\nN...,[Chorus]
...,...,...,...,...,...
956,blackbear,​if i could i would feel nothing,[Chorus],'Cause if I could I would feel nothing\r\nThat...,[Chorus]
957,blackbear,​if i could i would feel nothing,[Post-Chorus],"So let's just fuck, no strings attached\r\nLet...",[Post-Chorus]
958,blackbear,​if i could i would feel nothing,[Verse 2],"February, weather's scary, I need notes on how...",[Verse]
959,blackbear,​if i could i would feel nothing,[Chorus],'Cause if I could I would feel nothing\r\nThat...,[Chorus]


In [19]:
def generate_input_prompt(part_of_song: str, song_name: str, artist_name: str):
  return f"""Generate the {part_of_song} of a song called '{song_name}' by {artist_name}"""

In [23]:
test_df['prompt'] = test_df.apply(lambda row: generate_input_prompt(row['tag_2'], row['song'], row['artist']),axis=1)
test_df

,artist,song,tag,content,tag_2,prompt
0,5 Seconds of Summer,No Shame,[Verse 1: Luke],"Angel, with the gun in your hand\r\nPointin' m...",[Verse],Generate the [Verse] of a song called 'No Sham...
1,5 Seconds of Summer,No Shame,"[Pre-Chorus: Luke, Luke & Ashton]",Go on and light me like a cigarette\r\nEven if...,[Pre-Chorus],Generate the [Pre-Chorus] of a song called 'No...
2,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin'\r\nN...,[Chorus],Generate the [Chorus] of a song called 'No Sha...
3,5 Seconds of Summer,No Shame,[Verse 2: Luke],Lay me in the palm of your hand\r\nI'll give y...,[Verse],Generate the [Verse] of a song called 'No Sham...
4,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin'\r\nN...,[Chorus],Generate the [Chorus] of a song called 'No Sha...
...,...,...,...,...,...,...
956,blackbear,​if i could i would feel nothing,[Chorus],'Cause if I could I would feel nothing\r\nThat...,[Chorus],Generate the [Chorus] of a song called '​if i ...
957,blackbear,​if i could i would feel nothing,[Post-Chorus],"So let's just fuck, no strings attached\r\nLet...",[Post-Chorus],Generate the [Post-Chorus] of a song called '​...
958,blackbear,​if i could i would feel nothing,[Verse 2],"February, weather's scary, I need notes on how...",[Verse],Generate the [Verse] of a song called '​if i c...
959,blackbear,​if i could i would feel nothing,[Chorus],'Cause if I could I would feel nothing\r\nThat...,[Chorus],Generate the [Chorus] of a song called '​if i ...


In [27]:
sample = test_df.sample(50)
prompts = sample["prompt"]
songs = sample["content"]

In [ ]:
def generate_prediction(model,tokenizer,input_sequence):
    model_input = tokenizer(
        [input_sequence],
        return_tensors="pt",
        padding=False,
        max_length=256,
        truncation=True,
    ).input_ids.to(device)

    gen_kwargs = {
        "max_length": 256,
        "num_beams": 5,
        "num_return_sequences": 3, # Output the top 3 most probable sequences
    }

    outputs = model.generate(input_ids=model_input, **gen_kwargs)
    generated_sequences = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return generated_sequences

In [9]:
data = [
    [True, "[Verse]", "hola hola hola hola"],
    [False, "[Verse]", "adios adios adios"]
]

df = pd.DataFrame(data, columns=["is_generated", "tag", "song"])
df

,is_generated,tag,song
0,True,[Verse],hola hola hola hola
1,False,[Verse],adios adios adios


In [7]:
def generate_prompt(tag, song):
    return f"Evaluate from 0/10 coherence and quality of this {tag} of a pop song: {song}"

In [ ]:
df['d'] = df.apply(lambda x: some_func(a = x['a'], b = x['b'], c = x['c']), axis=1)


In [11]:
df["promt"] = df.apply(lambda x: generate_prompt(x["tag"], x["song"]), axis=1)
df

,is_generated,tag,song,promt
0,True,[Verse],hola hola hola hola,Evaluate from 0/10 coherence and quality of th...
1,False,[Verse],adios adios adios,Evaluate from 0/10 coherence and quality of th...
